In [1]:
library(XML)
ubase = "http://www.cherryblossom.org/"

#### From text
menURLs = 
  c("cb99m.htm", 
    "cb003m.htm", 
    "results/2001/oof_m.html",
    "results/2002/oofm.htm", 
    "results/2003/CB03-M.HTM",
    "results/2004/men.htm", 
    "results/2005/CB05-M.htm", 
    "results/2006/men.htm", 
    "results/2007/men.htm", 
    "results/2008/men.htm", 
    "results/2009/09cucb-M.htm",
    "results/2010/2010cucb10m-m.htm", 
    "results/2011/2011cucb10m-m.htm",
    "results/2012/2012cucb10m-m.htm")
####

#### Text URLS
urls = paste(ubase, menURLs, sep="")
urls[1:4]

# 1999: http://www.cherryblossom.org/cb99m.htm
# 2000: http://www.cherryblossom.org/cb003m.htm
# 2001: http://www.cherryblossom.org/results/2001/oof_m.html

#### Textbook Function
extractResTable =
  #
  # Retrieve data from web site, 
  # find the preformatted text,
  # and write lines or return as a character vector.
  #
  function(url = "http://www.cherryblossom.org/results/2009/09cucb-F.htm",
           year = 1999, sex = "male", file = NULL)
  {
    #added encoding for windows users who get an "A" symbol
    doc = htmlParse(url)    
    #doc = htmlParse(url, encoding="UTF-8")
    
    if (year == 2000) {
      # Get preformatted text from 4th font element
      # The top file is ill formed so the <pre> search doesn't work.
      ff = getNodeSet(doc, "//font")
      txt = xmlValue(ff[[4]])
      els = strsplit(txt, "\r\n")[[1]]
    }
    else if (year == 2009 & sex == "male") {
      # Get preformatted text from <div class="Section1"> element
      # Each line of results is in a <pre> element
      div1 = getNodeSet(doc, "//div[@class='Section1']")
      pres = getNodeSet(div1[[1]], "//pre")
      els = sapply(pres, xmlValue)
    }
    else {
      # Get preformatted text from <pre> elements
      pres = getNodeSet(doc, "//pre")
      txt = xmlValue(pres[[1]])
      els = strsplit(txt, "\r\n")[[1]]
#      els2 = strsplit(txt, "\n")[[1]]
    } 
    
    if (is.null(file)) return(els)
    # Write the lines as a text file.
    writeLines(els, con = file)
  }

# Skip over the first pass
#### Individual Input Components for Testing: 1999
url <- 'http://www.cherryblossom.org/results/1999/cb99m.html'
year <- 1999
sex <- "male"
file <- NULL
####

# Skip over the first pass
#### Individual Input Components for Testing: 2000
url <- 'http://www.cherryblossom.org/results/2000/Cb003m.htm'
year <- 2000
sex <- "male"
file <- NULL
####

#### Textbook example with (1) URL
df1 <- extractResTable(url = "http://www.cherryblossom.org/results/2000/Cb003m.htm", year = 2000, sex = "male", file = NULL)
df2 <- extractResTable(url = "http://www.cherryblossom.org/results/1999/cb99m.html", year = 1999, sex = "male", file = NULL)
#df3 <- extractResTableV2(url = "http://www.cherryblossom.org/results/1999/cb99m.html", year = 1999, sex = "male", file = NULL)

#### Textbook extraction of Male tables (results in an error)
years = 1999:2012
menTables = mapply(extractResTable, url = urls, year = years)
#names(menTables) = years # can't run b/c menTables hasn't been created
#sapply(menTables, length) # can't run b/c menTables hasn't been created

menTables <- list()
for(i in 1:length(years)){
  menTables[[i]] <- try(extractResTable(url=urls[i], year=years[i]))
}

# Let's go check out the first two URLs
urls[1] # [1] "http://www.cherryblossom.org/cb99m.htm"
urls[2] # [1] "http://www.cherryblossom.org/cb003m.htm"

#### Revised URLS
menURLsV2 = 
  c("results/1999/cb99m.html", #"cb99m.htm"
    "results/2000/Cb003m.htm", #"cb003m.htm"
    "results/2001/oof_m.html", #"results/2001/oof_m.html"
    "results/2002/oofm.htm", #"results/2002/oofm.htm"
    "results/2003/CB03-M.HTM", #"results/2003/CB03-M.HTM"
    "results/2004/men.htm", #"results/2004/men.htm"
    "results/2005/CB05-M.htm", #"results/2005/CB05-M.htm"
    "results/2006/men.htm", #"results/2006/men.htm"
    "results/2007/men.htm", #"results/2007/men.htm"
    "results/2008/men.htm", #"results/2008/men.htm"
    "results/2009/09cucb-M.htm", #"results/2009/09cucb-M.htm"
    "results/2010/2010cucb10m-m.htm", #"results/2010/2010cucb10m-m.htm"
    "results/2011/2011cucb10m-m.htm", #"results/2011/2011cucb10m-m.htm"
    "results/2012/2012cucb10m-m.htm" #"results/2012/2012cucb10m-m.htm"
  )
####

#### Revised URLS
urlsV2 = paste(ubase, menURLsV2, sep="")
urlsV2[1:4]

#### Modified textbook extraction of Male tables (results in 1999 having (1) record)
menTables = mapply(extractResTable, url = urlsV2, year = years)
names(menTables) = years
sapply(menTables, length)

#### Code to compare and contrast the format of two different years
substr(menTables$'1999', start = 1, stop = 100)
substr(menTables$'2000', start = 1, stop = 100)
menTables$'2000'[1:10]

#### Revised Function
extractResTableV2 =
  #
  # Retrieve data from web site, 
  # find the preformatted text,
  # and write lines or return as a character vector.
  #
  function(url = "http://www.cherryblossom.org/results/2009/09cucb-F.htm",
           year = 1999, sex = "male", file = NULL)
  {
    #added encoding for windows users who get an "A" symbol
    doc = htmlParse(url, encoding="UTF-8")
    
    if (year == 2000) {
      # Get preformatted text from 4th font element
      # The top file is ill formed so the <pre> search doesn't work.
      ff = getNodeSet(doc, "//font")
      txt = xmlValue(ff[[4]])
      els = strsplit(txt, "\r\n")[[1]]
    }
    else if (year == 2009 & sex == "male") {
      # Get preformatted text from <div class="Section1"> element
      # Each line of results is in a <pre> element
      div1 = getNodeSet(doc, "//div[@class='Section1']")
      pres = getNodeSet(div1[[1]], "//pre")
      els = sapply(pres, xmlValue)
    }
    else if (year == 1999 & sex == "male") { # have to add this else if statement
      # Get preformatted text from <pre> elements
      pres = getNodeSet(doc, "//pre")
      txt = xmlValue(pres[[1]])
      els = strsplit(txt, "\n")[[1]]   
    } 
    else {
      # Get preformatted text from <pre> elements
      pres = getNodeSet(doc, "//pre")
      txt = xmlValue(pres[[1]])
      els = strsplit(txt, "\r\n")[[1]]   
    } 
    
    if (is.null(file)) return(els)
    # Write the lines as a text file.
    writeLines(els, con = file)
  }

#### Corrected function to pull down Male tables with consistent format
menTablesV2 = mapply(extractResTableV2, url = urlsV2, year = years)
names(menTablesV2) = years
sapply(menTablesV2, length)

#### Confirmation that the 1999 and other years have consistent formatting
menTablesV2$'1999'[1:10]
menTablesV2[[2]][1:10]

#### Save the outputs
save(menTablesV2, file = "CBMenTextTables.rda")

#### Now we need to investigate the differences between the male and female result pages
# 2000
df_male_2000 <- extractResTableV2(url = "http://www.cherryblossom.org/results/2000/Cb003m.htm", year = 2000, sex = "male", file = NULL)
df_female_2000 <- extractResTableV2(url = "http://www.cherryblossom.org/results/2000/Cb003f.htm", year = 2000, sex = "female", file = NULL)

df_female_2000[1:10]
df_male_2000[1:10]

# 2006
df_male_2006 <- extractResTableV2(url = "http://www.cherryblossom.org/results/2006/men.htm", year = 2006, sex = "male", file = NULL)
df_female_2006 <- extractResTableV2(url = "http://www.cherryblossom.org/results/2006/women.htm", year = 2006, sex = "female", file = NULL)

df_female_2006[1:10]
df_male_2006[1:10]

######################################################################
# Miscellaneous Code

menTables <- list()
for(i in 1:length(years)){
  menTables[[i]] <- try(extractResTable(url=urlsV2[i], year=years[i]))
}

# Breaking down the extractResTableV2 for 1999 - Men
url <- urlsV2[1]
doc = htmlParse(url, encoding="UTF-8")
pres = getNodeSet(doc, "//pre")
txt = xmlValue(pres[[1]])
els = strsplit(txt, "\r\n")[[1]]
els = strsplit(txt, "\n")[[1]]

# Breaking down the extractResTableV2 for 2009 - Men
url <- urlsV2[11]
doc = htmlParse(url, encoding="UTF-8")
div1 = getNodeSet(doc, "//div[@class='Section1']")
pres = getNodeSet(div1[[1]], "//pre")
els = sapply(pres, xmlValue)



SyntaxError: invalid syntax (<ipython-input-1-efac8e6f4b08>, line 5)

In [2]:
ubase = "http://www.cherryblossom.org/"
menURLs = ["cb99m.htm", 
    "cb003m.htm", 
    "results/2001/oof_m.html",
    "results/2002/oofm.htm", 
    "results/2003/CB03-M.HTM",
    "results/2004/men.htm", 
    "results/2005/CB05-M.htm", 
    "results/2006/men.htm", 
    "results/2007/men.htm", 
    "results/2008/men.htm", 
    "results/2009/09cucb-M.htm",
    "results/2010/2010cucb10m-m.htm", 
    "results/2011/2011cucb10m-m.htm",
    "results/2012/2012cucb10m-m.htm"]
m_urls = [ubase + s for s in menURLs]
m_urls[1:4]

['http://www.cherryblossom.org/cb003m.htm',
 'http://www.cherryblossom.org/results/2001/oof_m.html',
 'http://www.cherryblossom.org/results/2002/oofm.htm']

In [3]:
ubase = "http://www.cherryblossom.org/"
womenURLs = ["results/1999/cb99f.html",
    "results/2000/Cb003f.htm",
    "results/2001/oof_f.html",
    "results/2002/ooff.htm",
    "results/2003/CB03-F.HTM",
    "results/2004/women.htm",
    "results/2005/CB05-F.htm",
    "results/2006/women.htm",
    "results/2007/women.htm",
    "results/2008/women.htm",
    "results/2009/09cucb-F.htm",
    "results/2010/2010cucb10m-f.htm",
    "results/2011/2011cucb10m-f.htm",
    "results/2012/2012cucb10m-f.htm"
]
f_urls = [ubase + s for s in womenURLs]
f_urls[13]

'http://www.cherryblossom.org/results/2012/2012cucb10m-f.htm'

In [4]:
import urllib.request
from bs4 import BeautifulSoup

URL = 'http://www.cherryblossom.org/results/2006/women.htm'

response = urllib.request.urlopen(URL)
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, "lxml")
#text = soup.get_text()
table = soup.find('pre').find(text=True)
raw_rows = [x.strip() for x in table.split('\n')]
#table_cols = raw_rows[6].split()
#table_cols[1] = table_cols[1] + table_cols[2]
#table_cols.remove("/TOT")
raw_rows[:10]

['',
 'Credit Union Cherry Blossom 10 Mile Road Race',
 'Washington, DC',
 'Sunday, April 2, 2006',
 "Women's Official Results Sorted By Gun Time",
 '',
 'Place Div/Tot  Num    Name                   Ag Hometown        Net Tim Gun Tim  Pace  S',
 '===== ======== ====== ====================== == ======================= =======  ===== =',
 '1   1/4069      4 Lidiya Grigoryeva      32 Russia            52:11   52:11#  5:14 !',
 '2   2/4069      6 Alevtina Ivanova       30 Russia            52:39   52:39#  5:16 !']

In [5]:
table_cols = raw_rows[6].split()
table_cols

['Place',
 'Div/Tot',
 'Num',
 'Name',
 'Ag',
 'Hometown',
 'Net',
 'Tim',
 'Gun',
 'Tim',
 'Pace',
 'S']

In [6]:
table_cols = raw_rows[6].split()
table_cols[6] = table_cols[6] + ' ' + table_cols[7]
table_cols[8] = table_cols[8] + ' ' + table_cols[9]
table_cols.remove("Tim")
table_cols.remove("Tim")
table_cols

['Place',
 'Div/Tot',
 'Num',
 'Name',
 'Ag',
 'Hometown',
 'Net Tim',
 'Gun Tim',
 'Pace',
 'S']

In [7]:
#table_cols = raw_rows[10]
#table_cols = list(table_cols.split())
table_cols = raw_rows[6].split()
table_cols[1] = table_cols[1] + table_cols[2]
#table_cols[7] = table_cols[7] + ' ' + table_cols[8]
#table_cols[9] = table_cols[9] + ' ' + table_cols[10]
table_cols.remove("/Tot")
#table_cols.remove("Tim")
#table_cols.remove("Tim")
table_cols

ValueError: list.remove(x): x not in list

In [11]:
import re
parsed_rows = []
for row in raw_rows[6:len(raw_rows[6:])+1]:
    parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s', row))
    if len(re.split(r'\s{2,}|(?<=[0-9])\s', row)) > 10:
        print(re.split(r'\s{2,}|(?<=[0-9])\s', row))
    #parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s', row))
#parsed_rows[2160:]

In [12]:
len(parsed_rows)

5434

In [13]:
import pandas as pd
import re

table = soup.find("pre").find(text=True)
raw_rows = [x.strip() for x in table.split('\n')]
table_cols = raw_rows[1].split()
table_cols[1] = table_cols[1] + table_cols[2]
table_cols.remove("/TOT")
parsed_rows = []
for row in raw_rows[3:len(raw_rows[3:])+1]:
    parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s', row))

ValueError: list.remove(x): x not in list

In [14]:
# ISSUE: Data entries are misplaced in columns due to missing values
df = pd.DataFrame(parsed_rows, columns=table_cols)
df

ValueError: 8 columns passed, passed data had 10 columns

In [ ]:
f_urls[1]

In [174]:
# scrape 1999
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

def scrape1999(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    text = soup.get_text()
    raw_rows = [x.strip() for x in text.split('\n')]
    table_cols = list(raw_rows[28].split())
    table_cols[1] = table_cols[1] + table_cols[2]
    table_cols.remove("/TOT")
    parsed_rows = []
    for row in raw_rows[30:2386]:
        parsed = re.split(r'\s{2,}|(?<=[0-9])\s', row) 
        if len(parsed) == len(table_cols):
            parsed_rows.append(parsed)
        else:
            adjusted_parsed = []
            adjusted_parsed.append(parsed[0])
            adjusted_parsed.append(float('NaN'))
            adjusted_parsed.append(parsed[1])
            adjusted_parsed.append(float('NaN'))
            adjusted_parsed.extend(parsed[2:5])
            parsed_rows.append(adjusted_parsed)
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df
    
f1999 = scrape1999(f_urls[0])
f1999.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 7 columns):
PLACE       2356 non-null object
DIV/TOT     2300 non-null object
NAME        2356 non-null object
AG          2300 non-null object
HOMETOWN    2356 non-null object
TIME        2356 non-null object
PACE        2356 non-null object
dtypes: object(7)
memory usage: 129.0+ KB


In [15]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape2000(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    text = soup.get_text()
    raw_rows = [x.strip() for x in text.split('\n')]
    table_cols = list(raw_rows[10].split())
    table_cols[1] = table_cols[1] + table_cols[2]
    table_cols[7] = table_cols[7] + ' ' + table_cols[8]
    table_cols[9] = table_cols[9] + ' ' + table_cols[10]
    table_cols.remove("/TOT")
    table_cols.remove("TIM")
    table_cols.remove("TIM")
    parsed_rows = []
    for row in raw_rows[12:(12+2166)]:
        parsed = re.split(r'\s{2,}|(?<=[0-9])\s', row)
        if len(parsed) > len(table_cols):
            parsed[5] = parsed[5] + ' ' + parsed[6]
            del parsed[6]
        parsed_rows.append(parsed)
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2000 = scrape2000(f_urls[1])
f2000
# Parsing Issue: Net Time is grouped under Gun Time

,PLACE,DIV/TOT,NUM,NAME,AG,HOMETOWN,GUN TIM,NET TIM
0,1,1/1503,7005,Teresa Wanjiku,25,Kenya,55:53#,55:50
1,2,2/1503,7001,Jane Omoro,27,Kenya,55:58#,55:56
2,3,3/1503,7008,Jane Ngotho,32,Kenya,56:04#,56:01
3,4,4/1503,7007,Nuta Olaru,29,Romania,56:07#,56:06
4,5,5/1503,7014,Salena Chirchir,31,Kenya,57:34#,57:31
...,...,...,...,...,...,...,...,...
2161,2162,1502/1503,8702,Tina Werking,30,Bethesda MD,2:23:25,2:20:20
2162,2163,60/60,10805,Karen Deck,56,York PA,2:24:31,2:24:31
2163,2164,180/180,8703,Kathleen Martin,45,Rockville MD,2:27:41,2:24:35
2164,2165,1503/1503,8996,Kathy Obert,38,Alexandria VA,2:38:27,2:38:27


In [16]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape2001(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = ['Place', 'Num', 'Name', 'Ag', 'Hometown', 'Net', 'Gun']
    parsed_rows = []
    for row in raw_rows[3:len(raw_rows[3:])+2]:
        parsed = re.split(r'\s{2,}|(?<=[0-9])\s', row)
        if len(parsed) > len(table_cols):
            parsed[4] = parsed[4] + ' ' + parsed[5]
            del parsed[5]
        parsed_rows.append(parsed)
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df
f2001 = scrape2001(f_urls[2])
f2001
# Parsing Issue: line 2968, age is grouped under name

,Place,Num,Name,Ag,Hometown,Net,Gun
0,1,6002,Elana MEYER,34,Rep Of S.africa,52:15,52:16#
1,2,6004,Lydia GRIGORIEVA,27,Russia,53:12,53:15#
2,3,6019,Eyerusalem KUMA,20,Ethiopia,53:16,53:19#
3,4,6007,Milena GLUSAC,25,Usa,53:33,53:34#
4,5,6012,Olga KOVPOTINA,31,Russia,54:01,54:03#
...,...,...,...,...,...,...,...
2967,2968,7227,Joanna LANZIROTTI,28,Woodbridge VA,2:28:53,2:28:53
2968,2969,9626,Stephanie DANNEMILLER 41,Arlington VA,2:18:34,2:29:22,None
2969,2970,9026,Alicia HOSMER,35,Bethesda MD,2:30:52,2:30:52
2970,2971,6658,Kathy OBERT,39,Alexandria VA,2:35:17,2:35:17


In [17]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape2002(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[1].split()
    parsed_rows = []
    for row in raw_rows[3:len(raw_rows[3:])+1]:
        parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s', row))
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df
f2002 = scrape2002(f_urls[3])
f2002

,Place,Num,Name,Ag,Hometown,Net,Gun
0,1,6005,Luminita TALPOS,29,Rom,52:50,52:50
1,2,6003,Teyba ERKASSO,20,Eth,52:53,52:55
2,3,6007,Sylvia MOSQUEDA,35,Usa,53:14,53:17
3,4,6022,Teresa WANJIKU,27,Ken,53:36,53:36
4,5,6020,Marla RUNYAN,33,Usa,53:37,53:37
...,...,...,...,...,...,...,...
3329,3330,10208,Ann HICKEY SHANKROFF,31,Falls Church VA,2:20:40,2:26:55
3330,3331,11140,Nanette VARIAS,48,Fairfax VA,2:25:49,2:29:46
3331,3332,7045,Joyce KIRKSEY,41,Alexandria VA,2:30:08,2:30:08
3332,3333,8016,Gail SUMMERS,39,Lafayette IN,2:38:58,2:38:58


In [34]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape2003(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[1].split()
    table_cols[1] = table_cols[1] + table_cols[2]
    table_cols[7] = table_cols[7] + ' ' + table_cols[8]
    table_cols[9] = table_cols[9] + ' ' + table_cols[10]
    table_cols.remove("/Tot")
    table_cols.remove("Tim")
    table_cols.remove("Tim")
    parsed_rows = []
    for row in raw_rows[3:len(raw_rows[3:])]:
        parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s',row))
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2003 = scrape2003(f_urls[4])
f2003
f2003.dtypes
f2003['Ag'] = f2003.Ag.astype(int)
f2003.dtypes

Place       object
Div/Tot     object
Num         object
Name        object
Ag           int64
Hometown    object
Gun Tim     object
Net Tim     object
dtype: object

In [19]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape2004(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[6].split()
    table_cols[1] = table_cols[1] + table_cols[2]
    table_cols.remove("/Tot")
    parsed_rows = []
    for row in raw_rows[8:]:
        parsed = re.split(r'\s{2,}|(?<=[0-9])\s', row)
        if len(parsed) > len(table_cols):
            parsed[5] = parsed[5] + ' ' + parsed[6]
            del parsed[6]
        parsed_rows.append(parsed)
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2004 = scrape2004(f_urls[5])
f2004

,Place,Div/Tot,Num,Name,Ag,Hometown,Net,Gun
0,1,1/2902,4,Isabella Ochichi,24,KEN,52:06,52:07
1,2,2/2902,36,Catherine Ndereba,31,KEN,52:58,53:00
2,3,3/2902,16,Victoria Klimina,28,RUS,53:08,53:08
3,4,4/2902,20,Tatyana Petrova,20,RUS,53:28,53:31
4,5,5/2902,2,Olga Romanova,23,RUS,53:44,53:45
...,...,...,...,...,...,...,...,...
3894,3895,260/261,12378,Debbie Melvin,49,Woodbridge VA,2:40:52,2:40:52
3895,3896,2901/2902,12373,Stacy Adams,25,Dumfries VA,2:44:51,2:44:51
3896,3897,408/408,880,Kathleen Obert,42,Alexandria VA,2:48:11,2:48:11
3897,3898,261/261,12379,Fran Hoffman,45,Stafford VA,2:53:56,2:53:56


In [20]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape2005(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[6].split()
    table_cols[1] = table_cols[1] + table_cols[2]
    table_cols.remove("/Tot")
    parsed_rows = []
    for row in raw_rows[8:(8+4333)]:
        parsed = re.split(r'\s{2,}|(?<=[0-9])\s', row)
        if len(parsed) > len(table_cols):
            parsed[4] = parsed[4] + ' ' + parsed[5]
            del parsed[5]
        parsed_rows.append(parsed)
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2005 = scrape2005(f_urls[6])
f2005

,Place,Div/Tot,Name,Ag,Hometown,Net,Gun,Pace
0,1,1/3146,Nuta Olaru,34,Romania,52:01,52:01,5:13
1,2,2/3146,Alvetina Ivanova,29,Russia,53:17,53:17,5:20
2,3,3/3146,Tetyana Hladyr,29,Ukraine,54:16,54:16,5:26
3,4,4/3146,Aurica Buia,35,Romania,54:57,54:57,5:30
4,5,5/3146,Sally Barsosio,27,Kenya,55:17,55:19,5:32
...,...,...,...,...,...,...,...,...
4328,4329,90/90,Sandra Carmichael,55,Manassas VA,2:50:28,2:50:28,17:03
4329,4330,502/502,Pravina Anikhindi,43,Gaithersburg MD,2:49:16,2:51:27,17:09
4330,4331,216/217,Karyn Hierl,51,Bethlehem PA,2:47:42,2:55:21,17:33
4331,4332,217/217,Elaine Freedman,51,Silver Spring MD,2:48:23,2:56:39,17:40


In [21]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape2006(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[6].split()
    table_cols[6] = table_cols[6] + ' ' + table_cols[7]
    table_cols[8] = table_cols[8] + ' ' + table_cols[9]
    table_cols.remove("Tim")
    table_cols.remove("Tim")
    parsed_rows = []
    for row in raw_rows[8:(8+5435)]:
        parsed = re.split(r'\s{2,}|(?<=[0-9])\s', row)
        parsed_rows.append(parsed)
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2006 = scrape2006(f_urls[7])
f2006

,Place,Div/Tot,Num,Name,Ag,Hometown,Net Tim,Gun Tim,Pace,S
0,1,1/4069,4,Lidiya Grigoryeva,32,Russia,52:11,52:11#,5:14,!
1,2,2/4069,6,Alevtina Ivanova,30,Russia,52:39,52:39#,5:16,!
2,3,3/4069,10,Lidia Simon,32,Romania,55:18,55:18#,5:32,!
3,4,4/4069,2,Luminita Talpos,33,Romania,55:33,55:33#,5:34,!
4,5,5/4069,18,Turena Johnson Lane,30,United States,55:42,55:42#,5:35,!
...,...,...,...,...,...,...,...,...,...,...
5430,5431,4066/4069,17514,Cheryl McCarthy,38,Roanoke,2:33:53,2:42:58,16:18,None
5431,5432,4067/4069,11175,Emily Whichard,24,Roanoke,2:33:53,2:42:58,16:18,None
5432,5433,4068/4069,16323,Angela Kleis,32,Centreville,2:35:47,2:43:13,16:20,None
5433,5434,227/227,9183,Helene Murtha,50,Baltimore,2:34:29,2:43:53,16:24,None


In [22]:
def scrape_2007(URL):
    response = urllib.request.urlopen(URL)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[5].split()
    table_cols[1] = table_cols[1] + table_cols[2]
    table_cols.remove("/Tot")
    parsed_rows = []
    for row in raw_rows[7:(7+5690)]:
        parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s', row))
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2007 = scrape_2007(f_urls[8])
f2007

,Place,Div/Tot,Num,Name,Ag,Hometown,Time,Pace,S,Split
0,1,1/728,4,Teyba Erkesso,24,Ethiopia,51:44#,5:11,!,32:06
1,2,2/728,16,Tatyana Petrova,23,Russia Germant.,52:58#,5:18,!,32:47
2,3,1/1083,8,Kathy Butler,33,United Kingdom,53:26#,5:21,!,33:02
3,4,3/728,28,Magdalene Makunzi,24,Kenya,53:45#,5:23,!,33:02
4,5,2/1083,12,Lidia Simon,33,Romania,53:52#,5:24,!,33:20
...,...,...,...,...,...,...,...,...,...,...
5685,5686,519/519,16162,Janet Nagy,41,Alexandria VA,#,1:36:21,None,None
5686,5687,728/728,1199,Alice Koethe,24,Washington DC,#,1:37:30,None,None
5687,5688,257/257,16506,Kim Reecher,50,Hagerstown MD,#,1:38:45,None,None
5688,5689,115/115,15681,Julia Lara,55,Washington DC,#,1:47:53,None,None


In [175]:
# scrape 2008
# def column_converter(df):
#     for col in df.columns:
#         if col in ['Place', 'Num', 'Age']:
#             df[col] = df[col].astype(str)
#         elif col in ['Name', 'Hometown']:
#             df[col] = df[col].astype(str)
#         elif col in ['Net', 'Gun', 'Time', 'Pace']:
#     return df

def scrape_2008(URL):
    response = urllib.request.urlopen(URL)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[6].split()
    table_cols[1] = 'Div/Tot'
    table_cols[7] = '5 Mi'
    table_cols[8] = '5 Mi Pace'
    table_cols[9] = '10 Km'
    table_cols[10] = '10 Km Pace'
    table_cols.remove('/Tot')
    table_cols.remove('Km')
    table_cols.remove('Pace')

    parsed_rows = []
    for row in raw_rows[8:(8+8323)]:
        row_contents = re.split(r'\s{2,}|(?<=[0-9])\s', row)
        if len(row_contents) == 12:
            parsed_rows.append(row_contents)
        else:
            modified_row = row_contents[0:6] + [0, 0, 0, 0] + row_contents[-2:]
            parsed_rows.append(modified_row)
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    #column_converter(df)
    return df

f2008 = scrape_2008(f_urls[9])
f2008[1:2]

f2008.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6398 entries, 0 to 6397
Data columns (total 12 columns):
Place         6398 non-null object
Div/Tot       6398 non-null object
Num           6398 non-null object
Name          6398 non-null object
Ag            6398 non-null object
Hometown      6398 non-null object
5 Mi          6397 non-null object
5 Mi Pace     6397 non-null object
10 Km         6397 non-null object
10 Km Pace    6397 non-null object
Time          6397 non-null object
Pace          6397 non-null object
dtypes: object(12)
memory usage: 599.9+ KB


In [90]:
def scrape_2009(URL):
    response = urllib.request.urlopen(URL)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[6].split()
    table_cols[1] = table_cols[1] + table_cols[2]
    table_cols[8] = table_cols[7] + table_cols[8]
    table_cols[9] = table_cols[9] + table_cols[10]
    table_cols.remove("/Tot")
    table_cols.remove("Gun")
    table_cols.remove("Tim")
    parsed_rows = []
    for row in raw_rows[8:(8+8323)]:
        parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s', row))
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2009 = scrape_2009(f_urls[10])
f2009

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8323 entries, 0 to 8322
Data columns (total 10 columns):
Place       8323 non-null object
Div/Tot     8323 non-null object
Num         8323 non-null object
Name        8323 non-null object
Ag          8323 non-null object
Hometown    8323 non-null object
GunTim      8323 non-null object
NetTim      8321 non-null object
Pace        8240 non-null object
S           33 non-null object
dtypes: object(10)
memory usage: 650.4+ KB


In [25]:
def scrape_2010(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[6].split()
    table_cols[7] = table_cols[7] + table_cols[8]
    table_cols[9] = table_cols[9] + table_cols[10]
    table_cols[11] = table_cols[11] + table_cols[12]
    table_cols.remove("/Tot")
    table_cols.remove("Mile")
    table_cols.remove("Tim")
    table_cols.remove("Tim")
    parsed_rows = []
    for row in raw_rows[8:(8+8853)]:
        parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s', row))
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2010 = scrape_2010(f_urls[11])
f2010

,Place,Div,Num,Name,Ag,Hometown,5Mile,GunTim,NetTim,Pace,S
0,1,1/971,2,Lineth Chepkurui,23,Kenya,25:38,51:51,51:51#,5:12,!
1,2,2/971,28,Julliah Tinega,24,Kenya,25:41,52:40,52:39#,5:16,!
2,3,3/971,6,Belainesh Zemedkun,22,Ethiopia,26:06,53:22,53:22#,5:21,!
3,4,4/971,30,Misker Demessie,23,Ethiopia,27:11,54:37,54:37#,5:28,!
4,5,1/1810,18,Kelly Jaske,33,Portland OR,27:10,54:40,54:40#,5:28,!
...,...,...,...,...,...,...,...,...,...,...,...
8848,8849,561/561,3876,Amita Goel,45,Centreville VA,1:13:31,2:42:58,2:29:50,14:59,None
8849,8850,1809/1810,18138,Heather Gunn,34,Gaithersburg MD,2:35:24,2:35:24,15:33,None,None
8850,8851,64/64,18608,Sook Lee,64,Potomac MD,2:36:05,2:36:05,15:37,None,None
8851,8852,1810/1810,14443,Vivian Lei,30,Arlington VA,1:18:43,2:36:31,2:36:31,15:40,None


In [26]:
def scrape_2011(URL):
    response = urllib.request.urlopen(URL)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[6].split()
    table_cols[1] = table_cols[1] + table_cols[2]
    table_cols[7] = table_cols[7] + ' ' + table_cols[8]
    table_cols[10] = table_cols[10] + ' ' + table_cols[11]
    table_cols.remove("/Tot")
    table_cols.remove("Mile")
    table_cols.remove("Tim")
    parsed_rows = []
    for row in raw_rows[8:(8+9030)]:
        parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s', row))
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2011 = scrape_2011(f_urls[12])
f2011
# Parsing Issue: row 9021, 5 mile time is grouped under Hometown

,Place,Div/Tot,Num,Name,Ag,Hometown,5 Mile,Time,Net Tim,Pace,S
0,1,1/2706,14,Julliah Tinega,25,Kenya,54:02,54:02,5:25,!,None
1,2,1/937,16,Risper Gesabwa,22,Kenya,27:17,54:03,54:03,5:25,!
2,3,1/1866,48,Tgist Tufa,30,Ethiopia,27:17,54:13,54:13,5:26,!
3,4,2/1866,44,Alemtsehay Misganaw,30,Ethiopia,27:17,55:17,55:17,5:32,!
4,5,2/2706,24,Claire Hallissey,28,United Kingdom,28:01,56:17,56:17,5:38,!
...,...,...,...,...,...,...,...,...,...,...,...
9025,9026,2705/2706,17287,Erin Grandstaff,28,Chevy Chase MD,1:16:38,2:24:45,2:24:45,14:29,None
9026,9027,1265/1265,14703,Catherine Hanmer,36,Astoria NY,1:16:40,2:39:20,2:26:45,14:41,None
9027,9028,2706/2706,17967,Kat Silvia,27,Lisle IL,1:07:21,2:35:01,2:26:51,14:42,None
9028,9029,1866/1866,18825,Tram Anh Tran,30,Fairfax VA,1:10:49,2:32:08,2:30:34,15:04,None


In [27]:
def scrape_2012(URL):
    response = urllib.request.urlopen(URL)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("pre").find(text=True)
    raw_rows = [x.strip() for x in table.split('\n')]
    table_cols = raw_rows[6].split()
    table_cols[1] = table_cols[1] + table_cols[2]
    table_cols.remove("/Tot")
    table_cols.remove("S")
    parsed_rows = []
    for row in raw_rows[8:(8+9729)]:
        parsed_rows.append(re.split(r'\s{2,}|(?<=[0-9])\s', row))
    df = pd.DataFrame(parsed_rows, columns = table_cols)
    return df

f2012 = scrape_2012(f_urls[13])
f2012
# Parsing Issue: age is grouped under name

,Place,Div/Tot,Num,Name,Ag,Hometown,5,Mile,Time,Pace
0,1,1/2781,2,Jelliah Tinega,26,Kenya,26:48,54:02,5:25,!
1,2,2/2781,24,Malika Mejdoub,29,Ethiopia,27:09,54:24,5:27,!
2,3,1/2228,22,Yihunlish Delelecha,30,Ethiopia,27:09,54:33,5:28,!
3,4,3/2781,14,Agnieszka Ciolek,26,Poland,27:30,54:36,5:28,!
4,5,4/2781,4,Claire Hallissey,29,Arlington VA,27:30,54:37,5:28,!
...,...,...,...,...,...,...,...,...,...,...
9724,9726,973/974,19016,Khristina Nava,40,Fort Meade MD,1:14:03,2:33:11,15:20,None
9725,9727,2228/2228,8258,Geneva Dixon,31,Manassas Park VA,1:14:26,2:36:03,15:37,None
9726,9728,236/236,23195,Veronica Eligan,55,Mitchellville MD,1:28:21,2:36:45,15:41,None
9727,9729,974/974,17855,Denise Bobba,40,Herndon VA,1:15:18,2:36:54,15:42,None


In [ ]:
f2007 = scrape_2007(f_urls[8])
f2007

In [ ]:
for row in parsed_rows:
    if len(row) > 12:
        print(row)

In [ ]:
f2007 = scrape_2009(f_urls[8])
f2007

In [ ]:
raw_rows